In [10]:
import glob
import sys
import time
import os.path
import copy
import numpy as np
import healpy
import fitsio
from multiprocessing import Pool

#from .utils import userprint

import glob
from matplotlib import pyplot as plt 
from astropy.io import fits
from astropy.table import Table, join, vstack


In [11]:
def userprint(*args, **kwds):
    """Defines an extension of the print function.
    Args:
        *args: arguments passed to print
        **kwargs: keyword arguments passed to print
    """
    print(*args, **kwds)
    sys.stdout.flush()

    
#line 1439 in master/py/picca/io.py
 
    
def read_deltas(in_dir,
                nside,
                lambda_abs,
                alpha,
                z_ref,
                cosmo,
                max_num_spec=None,
                no_project=False,
                from_image=None,
                nproc=None):
    
    files = []
    in_dir = os.path.expandvars(in_dir)
    if from_image is None or len(from_image) == 0:
        if len(in_dir) > 8 and in_dir[-8:] == '.fits.gz':
            files += sorted(glob.glob(in_dir))
        elif len(in_dir) > 5 and in_dir[-5:] == '.fits':
            files += sorted(glob.glob(in_dir))
        else:
            files += sorted(glob.glob(in_dir + '/*.fits') + glob.glob(in_dir +
                                                               '/*.fits.gz'))
    else:
        for arg in from_image:
            if len(arg) > 8 and arg[-8:] == '.fits.gz':
                files += sorted(glob.glob(arg))
            elif len(arg) > 5 and arg[-5:] == '.fits':
                files += sorted(glob.glob(arg))
            else:
                files += sorted(glob.glob(arg + '/*.fits') + glob.glob(arg +
                                                                '/*.fits.gz'))
    files = sorted(files)

    arguments = [(f, from_image) for f in files]
    pool = Pool(processes=nproc)
    results = pool.starmap(read_delta_file, arguments)
    pool.close()

    deltas = []
    num_data = 0
    for delta in results:
        if delta is not None:
            deltas += delta
            num_data = len(deltas)
            if (max_num_spec is not None) and (num_data > max_num_spec):
                break

    # truncate the deltas if we load too many lines of sight
    if max_num_spec is not None:
        deltas = deltas[:max_num_spec]
        num_data = len(deltas)

    userprint("\n")
    
    
    # compute healpix numbers
    phi = [delta.ra for delta in deltas]
    theta = [np.pi / 2. - delta.dec for delta in deltas]
    healpixs = healpy.ang2pix(nside, theta, phi)
    if healpixs.size == 0:
        raise AssertionError('ERROR: No data in {}'.format(in_dir))

    data = {}
    z_min = 10**deltas[0].log_lambda[0] / lambda_abs - 1.
    z_max = 0.
    for delta, healpix in zip(deltas, healpixs):
        z = 10**delta.log_lambda / lambda_abs - 1.
        z_min = min(z_min, z.min())
        z_max = max(z_max, z.max())
        delta.z = z
        if not cosmo is None:
            delta.r_comov = cosmo.get_r_comov(z)
            delta.dist_m = cosmo.get_dist_m(z)
        delta.weights *= ((1 + z) / (1 + z_ref))**(alpha - 1)

        if not no_project:
            delta.project()

        if not healpix in data:
            data[healpix] = []
        data[healpix].append(delta)

    return data, num_data, z_min, z_max

In [12]:
filename='/global/u1/s/sgontcho/desi/desi-everest/Delta_LYA/redo-metadata/Deltas/delta-1.fits.gz' 
l_ref=1216
hdul = fits.open(filename)
Deltas=np.array([])
Redshifts=np.array([])
RAs=np.array([])
DECs=np.array([])
datalist=[]
for m in range(0,173):
    
    data = Table(hdul[m].data)
    for row in range(len(data)):
        Deltas=np.append(Deltas,data.field(1)[row])
        RAs=np.append(RAs,hdul[m].header['RA'])
        DECs=np.append(DECs,hdul[m].header['DEC'])
        loglam=data.field(0)[row]
        l_obs=10**(loglam)
        z=(l_obs-l_ref)/l_ref
        Redshifts=np.append(Redshifts,z)
        

for m in range (len(Deltas)-1):
    datalist.append([RAs[m],DECs[m],Redshifts[m],Deltas[m]])                               
print(hdul[1].header)                  


XTENSION= 'BINTABLE'           / binary table extension                         BITPIX  =                    8 / 8-bit bytes                                    NAXIS   =                    2 / 2-dimensional binary table                     NAXIS1  =                   32 / width of table in bytes                        NAXIS2  =                  177 / number of rows in table                        PCOUNT  =                    0 / size of special data area                      GCOUNT  =                    1 / one data group (required keyword)              TFIELDS =                    4 / number of fields in each row                   TTYPE1  = 'LOGLAM  '           / label for field   1                            TFORM1  = 'D       '           / data format of field: 8-byte DOUBLE            TUNIT1  = 'log Angstrom'       / physical unit of field                         TTYPE2  = 'DELTA_BLIND'        / label for field   2                            TFORM2  = 'D       '           / data fo

In [5]:
deltatable = np.array(datalist)

In [12]:
deltatable

array([[  2.42055889,   1.4284102 ,   1.96052632, -37.07653328],
       [  2.42055889,   1.4284102 ,   1.96257208, -50.52021237],
       [  2.42055889,   1.4284102 ,   1.96461926, -17.28094307],
       ...,
       [  2.34039325,   1.46710695,   2.83326572,   0.09445987],
       [  2.34039325,   1.46710695,   2.83591457,   0.10167317],
       [  2.34039325,   1.46710695,   2.83856524,   0.1372551 ]])

In [13]:
col1 = fits.Column(name='ra', array=RAs, format="E")
col2 = fits.Column(name='dec', array=DECs, format="E")
col3 = fits.Column(name='z', array=Redshifts, format="E")
col4 = fits.Column(name='Delta', array=Deltas, format="E")

cols=fits.ColDefs([col1,col2,col3,col4])
hdu = fits.BinTableHDU.from_columns(cols)
hdu.writeto('/global/homes/s/schampat/Voids/Void_analysis/Data/deltatable.fits',overwrite=True)


In [14]:
filename='/global/homes/s/schampat/Voids/Void_analysis/Data/deltatable.fits'
hdul = fits.open(filename)
data = Table(hdul[1].data)
data




ra,dec,z,Delta
float32,float32,float32,float32
2.420559,1.4284102,1.9605263,-37.076534
2.420559,1.4284102,1.9625721,-50.520214
2.420559,1.4284102,1.9646193,-17.280943
2.420559,1.4284102,1.9666679,-36.681316
2.420559,1.4284102,1.9687178,15.435552
2.420559,1.4284102,1.9707693,-0.39883325
2.420559,1.4284102,1.9728222,-68.59262
2.420559,1.4284102,1.9748764,-25.844631
2.420559,1.4284102,1.976932,-40.438103
